In [3]:
import glob
from rasterio.merge import merge
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.windows import from_bounds
import matplotlib.pyplot as plt
import numpy as np



In [4]:
SWISSIMAGE_PATH = './dataset/base'
LULC_PATH = './dataset/lulc'
SWISSALTI3D_PATH = './dataset/alti'

OUTPUT_DATASET = './dataset'

In [5]:
def read_base_map_tile(east, north):
    img_files = glob.glob(f'{SWISSIMAGE_PATH}/swissimage-dop10_*_{east}-{north}_*.tif')
    if len(img_files) != 1:
        print('ERROR: ', img_files)

    with rasterio.open(img_files[0]) as dataset:
        tile_bounds = dataset.bounds
        r = dataset.read(1)
        g = dataset.read(2)
        b = dataset.read(3)
        rgb_base_image = []
        rgb_base_image = np.stack((r, g, b), axis=-1)

    return tile_bounds, rgb_base_image

def read_alti3d_tile(east, north, tile_bounds):
    alit3d_files = glob.glob(f'{SWISSALTI3D_PATH}/swissalti3d_*_{east}-{north}_*.tif')
    if len(alit3d_files) != 1:
        print('ERROR: ', alit3d_files)

    with rasterio.open(alit3d_files[0]) as dataset:
        window = from_bounds(tile_bounds.left, tile_bounds.bottom, tile_bounds.right, tile_bounds.top, transform=dataset.transform)
        tile = dataset.read(1, window=window)

    return tile

In [6]:
def get_base_image(east, north):
    img_files = glob.glob(f'{SWISSIMAGE_PATH}/swissimage-dop10_*_{east}-{north}_*.tif')
    if len(img_files) != 1:
        print('ERROR: ', img_files)
        return None

    file = img_files[0]
    src = rasterio.open(file)
    return src

def get_alti_image(east, north):
    img_files =glob.glob(f'{SWISSALTI3D_PATH}/swissalti3d_*_{east}-{north}_*.tif')
    if len(img_files) != 1:
        print('ERROR: ', img_files)
        return None

    file = img_files[0]
    src = rasterio.open(file)
    return src

def read_lulc_tile(east, north):
    lulc_tiles = glob.glob(f'{LULC_PATH}/{east}_{north}.npy')
    if len(lulc_tiles) != 1:
        print('ERROR: ', lulc_tiles, east, north)
        return np.full((40, 40), -9999)

    return np.load(lulc_tiles[0])

def get_bounds(east, north):
    img_files = glob.glob(f'{SWISSIMAGE_PATH}/swissimage-dop10_*_{east}-{north}_*.tif')
    file = img_files[0]
    with rasterio.open(file) as dataset:
        tile_bounds = dataset.bounds

    return tile_bounds


In [ ]:
east = 2504
north = 1159
size = 1

base_src_files_to_mosaic = []
alti_src_files_to_mosaic = []
lulc_tile_src_files_to_mosaic = []
for x in range(size):
    for y in range(size):
        east_l = east + x
        north_l = north + y

        bounds = get_bounds(east, north)

        base_src = get_base_image(east_l, north_l)
        alti_src = get_alti_image(east_l, north_l)
        lulc_tile = read_lulc_tile(bounds)

        if base_src != None:
            base_src_files_to_mosaic.append(base_src)
        if alti_src != None:
            alti_src_files_to_mosaic.append(alti_src)

        lulc_tile_src_files_to_mosaic.append(lulc_tile)

base_mosaic, base_out_trans = merge(base_src_files_to_mosaic)
alti_mosaic, alti_out_trans = merge(alti_src_files_to_mosaic)

plt.imsave(f'{OUTPUT_DATASET}/merged/base/{east}_{north}_{size}.png', np.transpose(base_mosaic, (1, 2, 0)))
plt.imsave(f'{OUTPUT_DATASET}/merged/alti/{east}_{north}_{size}.png', alti_mosaic[0], cmap='gray')